In [1]:
import torch
device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")

In [2]:
import json
from transformers import AutoTokenizer,RobertaModel
local_model_path = "/Users/renhonglow/Desktop/FYP/DTI/ChemBERTa-10M-MTR"

tokenizer = AutoTokenizer.from_pretrained(local_model_path)
model = RobertaModel.from_pretrained(local_model_path).to(device)
model.eval()

In [4]:
import os
import json
filepath= os.path.abspath('davis/ligands_can.txt')

with open(filepath,'r') as file:
    smile_dict =json.load(file)

In [7]:
smile_dict

In [8]:
print(len(smile_dict))

In [9]:
import os
filepath= os.path.abspath('kiba/ligands_iso.txt')

with open(filepath,'r') as file:
    smile_dict =json.load(file)

In [10]:
smile_dict

In [11]:
print(len(smile_dict))

In [7]:
i=0
max_length=0
for smile in smile_dict.values():
    print(f"Smiles {i}: {smile}，length: {len(smile)}")
    i=i+1
    current_length = len(smile)
    if current_length > max_length:
        max_length = current_length

print(f"Max SMILES length: {max_length}")

In [8]:
smiles = list()
for smile in smile_dict.values():
    smiles.append(smile)

last_hidden_state的形状的含义为(batch_size, sequence_length, hidden_size) <br/>
sequence_length 就是批次中最长的 token 序列长度

In [ ]:
input = tokenizer(smiles,padding=True,truncation=True,max_length=512,return_tensors="pt").to(device)
output = model(**input)
print(output.last_hidden_state.shape)

In [ ]:
cls_embedding = output.last_hidden_state[:, 0, :]
print(cls_embedding.shape)  # 输出形状应为 (batch_size, hidden_size)
mean_embedding = output.last_hidden_state.mean(dim=1)
print(mean_embedding.shape)  # 输出形状为 (batch_size, hidden_size)
combined_embedding = torch.cat([cls_embedding, mean_embedding], dim=-1)
print(combined_embedding.shape)  # 输出形状为 (batch_size, hidden_size * 2)

In [ ]:
features = output.last_hidden_state.cpu()
torch.save(features, 'ligands.pt')

In [9]:
import torch

features = torch.load('ligands_kiba.pt')
print(features.shape)

In [ ]:
token_embeddings = features
print(token_embeddings.shape)